## 	📘Apache Logs ETL Pipeline - Cybersecurity Analysis

This notebook parses and transforms Apache access log  data to identify suspicious activity, analyze request patterns and prepare cleaned datasets for Power BI dashboarding.

In [1]:
import pandas as pd
import re
from datetime import datetime

### 📁 Load and Parse Apache Access Logs

Using regex to extract fields like IP, timestamp, HTTP method, endpoint and status code from standard Apache log lines.

In [2]:
# Load the data and regex parsing for the data convert into csv and appropriate format


# define the log pattern
pattern=re.compile(r'(?P<ip>\S+) \S+ \S+ \[(?P<time>[^\]]+)\] "(?P<request>[^"]+)" (?P<status>\d{3}) (?P<size>\S+)')

#with log lines
with open(r"C:\Users\2247823\Downloads\apache_logs.log") as f:
    lines=f.readlines()
    
# parse lines
records=[]
for line in lines:
    match=pattern.match(line)
    if match:
        records.append(match.groupdict())
        
df=pd.DataFrame(records)
df.head()

,ip,time,request,status,size
0,83.149.9.216,17/May/2015:10:05:03 +0000,GET /presentations/logstash-monitorama-2013/im...,200,203023
1,83.149.9.216,17/May/2015:10:05:43 +0000,GET /presentations/logstash-monitorama-2013/im...,200,171717
2,83.149.9.216,17/May/2015:10:05:47 +0000,GET /presentations/logstash-monitorama-2013/pl...,200,26185
3,83.149.9.216,17/May/2015:10:05:12 +0000,GET /presentations/logstash-monitorama-2013/pl...,200,7697
4,83.149.9.216,17/May/2015:10:05:07 +0000,GET /presentations/logstash-monitorama-2013/pl...,200,2892


### 🧹Clean and Transform Data

- Convert timestamps
- Split request into method, endpoint
- Mark failed logins(HTTP 401/403)
- Extract day/hour

In [5]:
# Clean adn transform the data

# clean and extract the features

# convert time to datetime
df['time']=pd.to_datetime(df['time'], format="%d/%b/%Y:%H:%M:%S %z")

#Split request into method, endpoint, protocol
df[['method','endpoint','protocol']]=df['request'].str.split(" ",expand=True)

# convert status to numeric
df['status']=df['status'].astype(int)

# failed login detection (HTTP 401,403)
df['is_failed_login']=df['status'].isin([401, 403])


In [9]:
# Extract Time Features

df['hour']=df['time'].dt.hour
df['day']=df['time'].dt.day_name()

### 	📊 Aggregate Key Metrics

1. Failed logins by IP
2. Request count by hour
3. Hourly activity heatmap (day vs hour)

In [11]:
## Aggregate & Analyze

In [12]:
# IPs with most failed logins

ip_fails=(
    df[df['is_failed_login']]
    .groupby('ip')
    .size()
    .reset_index(name='failed_attempts')
    .sort_values(by='failed_attempts', ascending=False)
)

In [13]:
df.head()

,ip,time,request,status,size,method,endpoint,protocol,is_failed_login,hour,day
0,83.149.9.216,2015-05-17 10:05:03+00:00,GET /presentations/logstash-monitorama-2013/im...,200,203023,GET,/presentations/logstash-monitorama-2013/images...,HTTP/1.1,False,10,Sunday
1,83.149.9.216,2015-05-17 10:05:43+00:00,GET /presentations/logstash-monitorama-2013/im...,200,171717,GET,/presentations/logstash-monitorama-2013/images...,HTTP/1.1,False,10,Sunday
2,83.149.9.216,2015-05-17 10:05:47+00:00,GET /presentations/logstash-monitorama-2013/pl...,200,26185,GET,/presentations/logstash-monitorama-2013/plugin...,HTTP/1.1,False,10,Sunday
3,83.149.9.216,2015-05-17 10:05:12+00:00,GET /presentations/logstash-monitorama-2013/pl...,200,7697,GET,/presentations/logstash-monitorama-2013/plugin...,HTTP/1.1,False,10,Sunday
4,83.149.9.216,2015-05-17 10:05:07+00:00,GET /presentations/logstash-monitorama-2013/pl...,200,2892,GET,/presentations/logstash-monitorama-2013/plugin...,HTTP/1.1,False,10,Sunday


In [17]:
# Request  volume by hour

hourly_requests=(
    df.groupby('hour')
    .size()
    .reset_index(name='request_count')
)

In [18]:
# Heatmap Dataset: Hour x Day

heatmap_data=(
    df.groupby(['day', 'hour'])
    .size()
    .reset_index(name='request_count')
)

### 🗃️ Export Cleaned & Aggregate Data for Power BI

In [19]:
## Export  to CSV

df.to_csv("cleaned_apache_logs.csv", index=False)
ip_fails.to_csv("failed_logins_by_ip.csv",index=False)
hourly_requests.to_csv("hourly_requests.csv", index=False)
heatmap_data.to_csv("hourly_day_requests.csv",index=False)

###  ✅ Next Steps

- Load these CSVs into Power BI
- Build dashboard with bar chart, line chart, heatmap and filterable table